## NLP를 이용한 구직자를 위한 회사 추천 시스템 + 감성 분석기

---------------------------

### INTRO  

이 프로젝트는 10개 업종의 654개 회사의 재직자 65385 명이 작성한 회사 리뷰를 분석하여 구직자가 원하는 회사를 추천해주는 것을 목표로 합니다. 또한 어떤 회사들의 리뷰가 서로 비슷한지를 확인해 볼 것이며, 어떤 텍스트가 회사에 대해 긍/부정의 극성을 나타내는지 판단하는 감성 분석기를 제작할 것입니다.

---------------------------

### DATA

분석에 사용된 데이터는 기업 리뷰와 정보를 공유하는 소셜 미디어, 잡 플래닛에서 스크래핑을 통해 확보하였습니다. 스크래핑에 사용된 파이썬 라이브러리는 BeautifupSoup 과 Selenium 입니다.

 * https://www.jobplanet.co.kr/welcome/index
 
![picture/1.PNG](picture/1.PNG)

총 65385개의 리뷰를 수집하였으며, 각 리뷰에는 회사의 장점과 단점에 대한 서술이 포함되어 있습니다. 장점과 단점 리뷰를 하나의 columns으로 합쳤기에 총 문서의 길이는 65385 + 65385 인 130770 입니다.

---------------------------

### DATA PREPROCESSING

컴퓨터가 인간이 일상에서 사용하는 자연어를 인식하기 위해서는 자연어에 대한 전처리 과정이 필수적입니다. 특히 이번 데이터 분석에 사용된 리뷰 데이터의 경우 뉴스 텍스트나 법 조문 처럼 맞춤법이나 띄어쓰기의 정확도가 완벽하지 않기 때문에 전처리 과정에 많은 노력이 필요합니다. 예를들어 Bag of words 와 같은 Sparse matrix 로 텍스트 데이터의 벡터 공간을 표현하고자 하는 경우, 전처리가 제대로 되지 않았다면 '회사'라는 단어와 '회사는' ,' 회사도' 다른 단어로 분류 되어 비효율적인 연산이 이루어지게 되고 분석 알고리즘의 성능도 저하 됩니다. 데이터 전처리 과정은 다음과 같습니다.

 * normalize 를 통한 영어 단어, 특수문자 등의 노이즈 제거
 * CRF 알고리즘을 이용한 띄어쓰기 오류 교정
 * 단어 점수 추출 후 토크나이징 (L-Tokenizer)
 * 명사 추출
 
텍스트 데이터의 전처리는 비지도학습 기반 한국어 전처리 라이브러리인 soynlp를 사용 하였습니다. 이미 기존에 학습된 형태소 분석기를 사용해야하는 Konlpy 와 달리, soynlp 는 현재 자신이 가지고 있는 데이터를 직접 학습시켜 적용함으로써, 자기가 분석하고자 하는 데이터의 도메인에 특화 된 성능을 제공합니다.  
 
#### A. Normalize

![picture/3.PNG](picture/3.PNG)

#### B. 띄어쓰기 오류 교정

![picture/4.PNG](picture/4.PNG)

체리피킹일수도 있지만 해당 문장에서는 완벽하게 띄어쓰기가 교정되었습니다.

#### C. 토크나이즈

![picture/5.PNG](picture/5.PNG)

soynlp 의 토크나이저는 max score 토크나이저와 L-토크나이저가 있습니다.max score 토크나이저는 사전에 학습한 단어 점수를 이용하여, 문장에서 가장 단어일 확률이 높은순으로 sub word 들을 추출해가는 방식입니다. 예를들어 '연차사용은비교적자유로운편' 이라는 문장에서 max score 토크나이저는 사용, 비교, 자유라는 단어를 먼저 토큰화 한뒤, 그 다음에 연차, 은, 적, 로운, 편 을 토큰화 합니다. 이런 특성 때문에 max score 토크나이저는 띄어쓰기가 제대로 되어있지 않은 텍스트에도 적용이 가능합니다.

L-토크나이저는 동사, 명사, 형용사 등이 어절의 왼쪽 (L)에 등장하고 조사나 어미가 어절의 오른쪽 (R)에 등장하는 한국어의 어절 구조에 착안되었습니다. 문장의 왼쪽에서부터 하나의 음절씩 옮겨가며, 해당 음절에서 등장할 수 있는 단어의 조건부 확률을 구하는 식으로 토크나이즈를 실시하는 방식입니다. 이번에도 '연차 사용은 비교적 자유로운편' 이라는 문장을 예를 든다면, '연차'라는 단어의 첫 음절, '연' 다음에 등장할 수 있는 단어는 연봉, 연말, 연기, 연구 등 매우 많기 때문에, '연' 다음에 '연차'의 두번째 음절인 '차'가 등장할 조건부 확률은 매우 낮습니다. 하지만 다음 음절로 옮겨 '연 + 차' 라는 sub word 는 '연차' 라는 명확한 단어 정보를 제공합니다. 단어간 경계를 구분해야 한다는 점에서 L-토크나이저는 띄어쓰기가 잘 되어 있는 텍스트에 적용 가능합니다.

저는 띄어쓰기 오류 교정을 통해 텍스트 데이터의 띄어쓰기가 상당히 개선되었다고 판단하여 L-토크나이저를 통해 토크나이징을 하였습니다. 이렇게 토크나이징 된 텍스트 데이터는 리뷰의 긍/부정 서술 여부를 판단하는 감성 분석기 제작에 사용 됩니다.

#### D. 명사 추출

![picture/6.PNG](picture/6.PNG)

L 토크나이저로 토큰화 된 텍스트 데이터의 단어 중에서 명사 사전에 저장 된 단어들만을 추리는 과정을 진행하였습니다. 없다의 명사형 어미인 없음 까지 추출된 것을 확인할 수 있습니다. 명사만 추출 된 텍스트 데이터는 Doc2vec 알고리즘에 적용 됩니다. 명사 사전에는 전체 문서에 최소 10번 이상 등장한 명사만 저장되어 있습니다.

---------------------------

### FIND SIMILAR COMPANIES + CLUSTERING

유사한 문서 군집, 즉 유사한 리뷰를 가지고 있는 회사를 찾기 위해서 토큰화 되어 명사만을 추출한 데이터를 Doc2Vec 알고리즘에 학습하였습니다. 사용된 라이브러리는 Gensim 입니다. Doc2Vec으로 학습 될 임베딩 벡터는 200 차원입니다. 학습에 동원된 데이터의 회사는 654 개이므로 학습된 doc2vec 모델의 형태는 654 X 200 입니다. 단어의 앞뒤 맥락을 얼마나 판단하는지 정하는 window의 사이즈는 5로 설정했습니다. Doc2Vec 의 학습 모델은 skip-gram 을 선택했습니다. 

Doc2vec 모델을 학습시킨 후, most_similar() 메소드를 이용하여 회사명의 태그를 입력하면, 코사인 유사도로 가장 유사한 회사들의 목록을 출력할 수 있습니다. 또한 임베딩 벡터을 T-SNE 알고리즘을 이용하여 200개 차원을 2차원 벡터로 바꾼뒤, 이를 시각화 하였습니다.  
  
  
![picture/7.PNG](picture/7.PNG)
  
  
외국계 은행인 한국스탠다드차타드은행을 검색해본 결과, 한국스탠다드차타드은행 재직자들의 리뷰와 비슷한 리뷰를 가진 회사들은 한국씨티은행, 홍콩상하이은행,우리은행과 같은 은행들이었으며, 로버트보쉬,보쉬전장,지멘스, 디에이치엘, 한국지엠, 한국마이크로소프트와 같은 외국계기업들이었습니다. 동일 업종 여부, 외국계 기업의 여부에 따라서 각 회사의 리뷰 내용의 유사성이 결정된다고 판단할 수 있습니다.  
  
  
![picture/8.PNG](picture/8.PNG)
  
  
삼성전자와 가장 비슷한 회사로는 에스케이하이닉스로, 에스케이하이닉스는 삼성전자와 같이 반도체 제조에 강점을 가지고 있는 회사입니다. 그 외에도 삼성디스플레이, 엘지디스플레이, 삼성전자 와 같은 IT 제조사 들이 순위에 들었으며, 삼성카드, 삼성SDS, 삼성바이로직스 와 같은 삼성계열사들이 순위에 들었습니다. 하지만 삼성 계열사들이 가장 유사한 10개 회사에 속한 것의 원인이 삼성 그룹의 그룹문화에 의해 재직자들의 리뷰가 유사하게 작성된 것 때문인지, 아니면 삼성이라는 단어가 리뷰에서 자주 등장하게 된 것 때문인지는 파악이 되지 않습니다. 조금 더 확실한 해석을 위해서는, 전처리 과정에서 NER을 통해 회사의 명칭은 명사에서 제거하는 작업을 해야 할 것입니다.

![picture/9.PNG](picture/9.PNG)

![picture/10.PNG](picture/10.PNG)

k-means 알고리즘에 학습한 doc2vec 모델을 적용하여 기업 리뷰 별 클러스터링을 할 수도 있습니다.

---------------------------

### FIND MY COMPANIES

같은 단어라고 할 지라도 어떤 맥락에서 사용되는지에 따라 그 단어의 의미는 달라집니다. 예를 들어 회사의 장점을 설명하는 상황에서 업무라는 단어는 회사 업무의 긍정적인 점을 서술할 것이고, 단점을 말하기 위해 사용되는 업무라는 단어는 부정적인 상황을 설명할 것입니다. 

이번에는 자신이 원하는 회사의 장점과 피하고 싶은 단점을 고려하여 그에 맞는 회사를 제시해주는 과정을 진행할 것이므로, 리뷰의 장점과 단점에 대한 텍스트를 분리하여 각각 다른 doc2vec 모델을 만들었습니다. 

624개의 회사는 10개의 업종과 57개의 세부 업종으로 구분 되어있습니다. 사용자는 input() 함수를 이용하여 자신이 검색하고 싶은 회사의 업종과 세부업종을 선택한 뒤, 자신이 원하는 회사의 장점과 단점을 find_my_company() 함수에 입력합니다. 함수는 장점에 맞는 회사와 단점에 해당하는 회사를 사전에 학습시켜 놓은 장점 doc2vec 과 단점 doc2vec을 통해 찾습니다. 여왕-왕 = 여자인 것처럼, [원하는 장점을 가진 회사] - [피하고 싶은 단점을 가진 회사] 를 찾으면 본인이 원하는 장점을 가지고 있으면서, 피하고 싶은 단점을 가지지 않은 회사를 출력합니다. 해당 명령을 수행하는 메소드는 다음과 같습니다.

 * doc2vec_model.docvecs.most_similar(positive=[원하는 장점], negative=[피하고 싶은 단점])

![picture/12.PNG](picture/12.PNG)

![picture/11.PNG](picture/11.PNG)

오직 장점만으로 회사를 검색하고 싶다면 set_polarity 에서 'positive'를 입력하면 됩니다.

---------------------------

### SENTIMENT ANALSYS

이번에는 장점과 단점 리뷰를 토크나이즈한 데이터를 분석 모델에 학습시킨 후, 리뷰의 장점, 단점의 극성을 classification 하였습니다. Scikit-learn 에 내장된 베르누이 나이브 베이즈, 로지스틱 회귀, 다층퍼셉트론 모델을 이용하였으며, 트레인셋과 테스트 셋의 비율은 3:1 입니다. 각 모델의 Test accuracy 는 베르누이 나이브 베이즈 - 0.90799, 로지스틱 회귀 - 0.93161, MLP - 0.91213 으로 로지스틱 회귀가 가장 뛰어난 성능을 보였습니다. 

![picture/16.PNG](picture/16.PNG)

다음은 학습과 테스트에 동원되지 않은 새로운 문장을 분류한 결과 입니다.

![picture/13.PNG](picture/13.PNG)


같은 극성을 띄는 문장이라고 할 지라도, '너무' 라는 단어를 추가하여 극성의 정도를 강화한다면 긍/부정 판별에 대한 확률이 달라집니다. 

![picture/14.PNG](picture/14.PNG)  

또한, 긍정 서술에 부정적인 상황에 주로 쓰이는 '너무' 를 추가한다면 문장의 극성에 대한 판단이 긍정에서 부정으로 바뀌기도 합니다.

![picture/15.PNG](picture/15.PNG)  

---------------------------
